<center>ASMADUNNISA BEGUM SHAIK<H1>

In [10]:
import requests
import json
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Task 1: Fetch and store data
def fetch_and_store_data(api_key, year_range):
    base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
    dataset_id = "GHCND"
    location_id = "ZIP:80249"
    units = "standard"
    limit = 1000

    for year in range(year_range[0], year_range[1]):
        start_date = f"{year-1}-12-15"
        end_date = f"{year}-01-21"

        params = {
            'datasetid': dataset_id,
            'locationid': location_id,
            'units': units,
            'startdate': start_date,
            'enddate': end_date,
            'limit': limit
        }

        headers = {
            'token': api_key
        }

        try:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()

            data = response.json()
            file_name = f'data/winter_{year}-{year+1}.json'

            with open(file_name, 'w') as json_file:
                json.dump(data, json_file)

            print(f'Data for winter {year}-{year+1} saved to {file_name}')
        except Exception as e:
            print(f'Error fetching data for winter {year}-{year+1}: {str(e)}')

# Task 2: Combine JSON files into CSV with TMAX, TMIN, and TAVG
def combine_json_to_csv():
    all_data = []
    data_folder = 'data/'

    for year in range(2008, 2022):
        filename = f'{data_folder}winter_{year}-{year+1}.json'

        if os.path.exists(filename):
            with open(filename, 'r') as file:
                data = json.load(file)
                all_data.extend(data['results'])

    df = pd.DataFrame(all_data)
    df['date'] = pd.to_datetime(df['date'])
    df['TMAX'] = df[df['datatype'] == 'TMAX']['value']
    df['TMIN'] = df[df['datatype'] == 'TMIN']['value']
    df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2
    df[['date', 'TMAX', 'TMIN', 'TAVG']].to_csv(data_folder + 'all_data_max_min_avg.csv', index=False)

# Task 3: Compile data into CSV with TAVG for each date and year
def compile_data_to_csv():
    all_data = {}
    data_folder = 'data/'

    for year in range(2008, 2022):
        filename = f'{data_folder}winter_{year}-{year+1}.json'

        if os.path.exists(filename):
            with open(filename, 'r') as file:
                data = json.load(file)
                for entry in data['results']:
                    date = pd.to_datetime(entry['date']).strftime('%m-%d')

                    if date not in all_data:
                        all_data[date] = {}

                    all_data[date][f'{year}-{year+1}'] = entry['value']

    df = pd.DataFrame.from_dict(all_data, orient='index')
    df.sort_index(inplace=True)
    df.to_csv(data_folder + 'all_data_min.csv')

# Task 4: Functions to compute temperature averages
def average(df):
    return df['TAVG'].mean()

def average_warmest(df):
    return df['TMAX'].mean()

def average_coldest(df):
    return df['TMIN'].mean()


# Task 5: Generate heatmap using Seaborn
def generate_heatmap():
    try:
        df = pd.read_csv('data/all_data_max_min_avg.csv')
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df['date'].dt.strftime('%Y')
        df['day_month'] = df['date'].dt.strftime('%m-%d')
        pivot_df = df.pivot_table(index='year', columns='day_month', values='TAVG', aggfunc='mean')
        pivot_df = pivot_df.sort_index()

        if not pivot_df.empty:
            plt.figure(figsize=(15, 10))
            sns.heatmap(pivot_df, cmap='coolwarm', cbar_kws={'label': 'Average Temperature (°C)'})
            plt.title('Average Temperature (TAVG) Heatmap (2008-2022)')
            plt.xlabel('Day-Month')
            plt.ylabel('Year')
            plt.show()
        else:
            print("No valid data for generating the heatmap.")
    except Exception as e:
        print(f"Error generating heatmap: {str(e)}")

def coldest_new_years_temp():
    # Load the compiled CSV file
    df = pd.read_csv('data/all_data_min.csv')
    
    # Ensure you have a column in your DataFrame that represents the date
    # Adjust the column name accordingly
    if 'date' in df.columns:
        # Convert the date column to datetime
        df['date'] = pd.to_datetime(df['date'])
        
        # Filter the data for 1/1
        new_years_data = df[df['date'].dt.month == 1 & (df['date'].dt.day == 1)]
        
        if not new_years_data.empty:
            # Find the coldest average temperature
            coldest_avg_temp = new_years_data.min().min()
            return coldest_avg_temp
        else:
            print("No data available for New Year's Day.")
            return None
    else:
        print("The 'date' column is not present in the DataFrame.")
        return None

# Main function to orchestrate the workflow
def main():
    api_key = 'onMnQMZLPZARhFhBZKtOJAKVOpNwjSWU'
    year_range = (2008, 2022)

    fetch_and_store_data(api_key, year_range)
    combine_json_to_csv()
    compile_data_to_csv()

    all_data_max_min_avg = pd.read_csv('data/all_data_max_min_avg.csv')
    print("Average temperature:", average(all_data_max_min_avg))
    print("Average temperature of warmest days:", average_warmest(all_data_max_min_avg))
    print("Average temperature of coldest days:", average_coldest(all_data_max_min_avg))
    coldest_temp = coldest_new_years_temp()
    if coldest_temp is not None:
        print("The coldest average New Year's Day temperature in all the data was:", coldest_temp)

    generate_heatmap()

if __name__ == "__main__":
    main()


Data for winter 2008-2009 saved to data/winter_2008-2009.json
Data for winter 2009-2010 saved to data/winter_2009-2010.json
Data for winter 2010-2011 saved to data/winter_2010-2011.json
Data for winter 2011-2012 saved to data/winter_2011-2012.json
Data for winter 2012-2013 saved to data/winter_2012-2013.json
Data for winter 2013-2014 saved to data/winter_2013-2014.json
Data for winter 2014-2015 saved to data/winter_2014-2015.json
Data for winter 2015-2016 saved to data/winter_2015-2016.json
Data for winter 2016-2017 saved to data/winter_2016-2017.json
Data for winter 2017-2018 saved to data/winter_2017-2018.json
Data for winter 2018-2019 saved to data/winter_2018-2019.json
Data for winter 2019-2020 saved to data/winter_2019-2020.json
Data for winter 2020-2021 saved to data/winter_2020-2021.json
Data for winter 2021-2022 saved to data/winter_2021-2022.json
Average temperature: nan
Average temperature of warmest days: 43.0968992248062
Average temperature of coldest days: 17.1608527131782

In [2]:
import requests
import json
import pandas as pd
from datetime import datetime

# Function to make API call to NOAA and retrieve weather data
def fetch_weather_data(api_key, start_date, end_date):
    base_url = 'https://www.ncei.noaa.gov/data'
    endpoint = '/daily-summaries'
    params = {
        'datasetid': 'GHCND',
        'locationid': 'ZIP:80249',
        'units': 'standard',
        'startdate': start_date,
        'enddate': end_date,
        'limit': 1000
    }
    headers = {'token': api_key}
    response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
    return response.json()

# Function to calculate TAVG from TMAX and TMIN
def calculate_tavg(tmax, tmin):
    return (tmax + tmin) / 2

# Function to process weather data and save it to JSON files
def process_weather_data(api_key):
    years = range(2008, 2023)  # Including 2022
    for year in years:
        start_date = f'{year}-12-15'
        end_date = f'{year+1}-01-21'
        data = fetch_weather_data(api_key, start_date, end_date)
        with open(f'data/winter_{year}-{year+1}.json', 'w') as f:
            json.dump(data, f)

# Function to combine JSON files into a single CSV file with TMAX, TMIN, and TAVG
def combine_json_to_csv():
    data = []
    for year in range(2008, 2023):
        with open(f'data/winter_{year}-{year+1}.json', 'r') as f:
            weather_data = json.load(f)
            for entry in weather_data['results']:
                date = entry['date']
                tmax = entry.get('TMAX', None)
                tmin = entry.get('TMIN', None)
                tavg = calculate_tavg(tmax, tmin) if tmax is not None and tmin is not None else None
                data.append({'Date': date, 'TMAX': tmax, 'TMIN': tmin, 'TAVG': tavg})
    df = pd.DataFrame(data)
    df.to_csv('data/all_data_max_min_avg.csv', index=False)

# Function to filter and transform CSV data
def filter_and_transform_csv():
    df = pd.read_csv('data/all_data_max_min_avg.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month_Day'] = df['Date'].dt.strftime('%m-%d')
    df_pivot = df.pivot(index='Month_Day', columns='Year', values='TAVG')
    df_pivot.to_csv('data/all_data_min.csv')

# Function to compute average temperature from DataFrame
def average(df):
    return df.mean().mean()

# Function to compute average of minimum temperature from DataFrame
def average_warmest(df):
    return df.min().mean()

# Function to compute average of maximum temperature from DataFrame
def average_coldest(df):
    return df.max().mean()

# Main function to execute the tasks
def main():
    api_key = 'onMnQMZLPZARhFhBZKtOJAKVOpNwjSWU'  # Replace 'YOUR_API_KEY' with your actual NOAA API key
    process_weather_data(api_key)
    combine_json_to_csv()
    filter_and_transform_csv()
    # Loading data for further analysis
    df = pd.read_csv('data/all_data_min.csv', index_col='Month_Day')
    # Computing temperature averages
    avg_temp = average(df)
    avg_warmest = average_warmest(df)
    avg_coldest = average_coldest(df)
    print("Average Temperature:", avg_temp)
    print("Average of Warmest Temperatures:", avg_warmest)
    print("Average of Coldest Temperatures:", avg_coldest)

if __name__ == "__main__":
    main()


JSONDecodeError: Expecting value: line 1 column 1 (char 0)